<a href="https://colab.research.google.com/github/a24lorie/DeepLearningKeras/blob/UIMP/DL_MultiClasificaci%C3%B3n_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ejemplo de clasificación multiclase con Keras + TensorFlow

##1. Subir el conjunto de datos a nuestro Drive
Lo primero que vamos a hacer es subir el fichero de datos que nos hemos descargado de la web de la asignatura a nuestro Drive.

Para ello ejecutamos el siguiente trozo de código y seleccionamos el fichero adecuado.

In [0]:
from google.colab import files 
files.upload()

## 2. Importar librerías necesarias
Ahora vamos a instalar las librerías necesarias para dibujar la red gráficamente. Estas librerías no son imprescindibles, pero vamos a incorporarlas para utilizar una función de keras que nos permitirá ver de manera gráfica la red creada.

Después cargaremos las librerías que sí son imprescindibles.



In [0]:
#Instalar librerias para dibujar la red gráficamente
!pip install pydot
!apt-get install graphviz

#Cargar librerias necesarias
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import plot_model
from keras.utils import to_categorical

## 3. Creación de la red y funciónes auxiliares

En esta ocasión, vamos a organizar el código en funciones.

La primera función es **plot_history**, que como sucedía en el problema de regresión, la utilizaremos para dibujar la gráfica en la que se ve la evolución del error durante ele entrenamiento.

La segunda función se ocupa de i) crear la red, ii) seleccionar el optimizador e inicializar parámetros, y iii) realizar el aprendizaje mostrando gráficamente la evolución del error.

Cabe destacar en este apartado que la capa de salida tiene tantas celdas como número de clases. Además, fíjate que en la capa de salida, la función de activación es la función **softmax**. Una explicación detallada del por qué se usa esta función puedes encontrarla en los vídeos que hablan de la arquitectura de la red y de la clasificación multiclase.

Fíjate también en la función que se optimiza: **categorical_crossentropy**. Si consultas la ayuda, verás que es la función que se corresponde con la función de pérdida vista en los vídos para los problemas multiclase.


**Nota**: la creación de la red podría haberse sacado de esta función puesto que únicamente se necesita crear la primera vez. Sin embargo, para que los resultados sean reproducibles se necesita que los parámetros iniciales del modelo sean los mismos en cada ejecución. Podría haberse logrado usando varias funciones, pero hemos decidido sacrificar un poco la eficiencia en aras de aumetar la claridad.


 

In [0]:
def plot_history(history):
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('categorical_crossentropy')
    plt.plot(history.epoch, np.array(history.history['loss']),
             label='Train Loss')
    plt.legend()
    plt.ylim([0, max(history.history['loss'])])


def entrena(X, Y, learning_rate=0.001, batch_size=32, epochs=1000):
    """Función que crea la red, define el modelo y entrena"""

    """ SE CREA LA RED """
    # Fijar semillas de números aleatorios
    np.random.seed(2032)  
    tf.set_random_seed(2032)

    n_input = 4  # Celdas en la capa de entrada (features)
    n_hidden1 = 6  # Celdas en la primera capa oculta
    n_hidden2 = 5  # Celdas en la segunda capa oculta
    n_hidden3 = 4  # Celdas en la tercera capa oculta
    n_clases = 3  # Celdas en la capa de salida

    # Definir las capas del modelo
    model = Sequential()
    model.add(Dense(n_hidden1, input_dim=n_input, activation='relu', name="hidden1_layer"))
    model.add(Dense(n_hidden2, input_dim=n_hidden1, activation='relu', name="hidden2_layer"))
    model.add(Dense(n_hidden3, input_dim=n_hidden2, activation='relu', name="hidden3_layer"))
    model.add(Dense(n_clases, activation='softmax', name="output_layer"))

    # Como se puede ver, se crea un modelo secuencial donde cada capa añadida 
    # recibe como entrada la salida de la anterior.
    # La dimensión de la entrada siempre se define en la primera capa del modelo
    # (parámetro input_dim) => model.add(Dense(n_hidden1, input_dim=n_input, name="hidden1_layer"))

    # Existe también una forma "funcional" de crear modelos. En este caso sería:
    # in_layer = Input(shape =(n_input,))
    # hidden1_layer = Dense(n_hidden1, name="hidden1_layer",activation='relu')(in_layer)
    # hidden2_layer = Dense(n_hidden2, name="hidden2_layer",activation='relu')(hidden1_layer)
    # hidden3_layer = Dense(n_hidden3, name="hidden3_layer",activation='relu')(hidden2_layer)
    # out_layer = Dense(n_output, name="out_layer")(hidden3_layer)
    #
    # model = Model(inputs = [in_layer], outputs = [out_layer])

    """ SELECCIONAR OPTIMIZADOR E INICIALIZAR PARÁMETROS """
    # Definir el optimizador a utilizar (SGD, descenso del gradiente estocástico)
    sgd = keras.optimizers.SGD(lr=learning_rate)

    # Compilar el modelo indicando la función de pérdida (loss) y el optimizador
    model.compile(loss='categorical_crossentropy', optimizer=sgd)

    """ SE VISUALIZA EL MODELO """
    # Imprimir en modo texto finalmente el resumen/arquitectura de nuestro modelo
    # Esta información permite conocer el número de parámetros que se han de aprender
    model.summary()

    # Obtener un fichero con la representación gráfica del modelo
    plot_model(model, to_file='model.png')
    # Después de ejecutar esta línea se genera el fichero 'model.png'. Para poder
    # verlo, tienes que ir a la parte izquierda, acceder a la pestaña 'Archivos' y
    # 'actualizar', aparecerá entonces el archivo y podrás verlo (doble click sobre
    # el fichero 'model.png' o click con el botón derecho para descargarlo)

    """ SE ENTRENA """
    # Entrenar el modelo a partir de los ejemplos de TRAIN creados en el punto 2
    history = model.fit(X, Y, batch_size=batch_size, epochs=epochs, verbose=0)
    print("Máximo error:",max(np.array(history.history['loss'])))
    print("Mínimo error:",min(np.array(history.history['loss'])))
    plot_history(history)

## 4. Cargar el conjunto

El conjunto está en el fichero **iris.data.txt**. Contiene datos de 150 lirios pertenecientes a 3 clases diferentes. Cada lirio viene descrito por cuatro atributos. 

Lo primero que hacemos es cargar el conjuto en un dataframe de pandas y barajar los ejemplos.

Después almacenamos en una matriz (laX) la descripción de los lirios y en un vector (laY) la clase de los mismos.

La clase, tal y como nos viene en el conjunto de datos, no puede ser utilizada directamente en la red. Necesitamos su codificación **one_hot**. Para obtenerla, vamos a utilizar una función de keras que se llama **to_categorical** y que crea una columna para cada clase empezando siempre en el 0. Consulta la ayuda para entender cómo funciona.


In [0]:
# Se carga el conjunto de datos de los lirios. Cada lirio está descrito 
# mediante 4 atributos y viene acompañado de su clase.
# Fíjate que la clase está en la última columna y solo puede ser 0, 1 o 2.
df = pd.read_csv('iris.data.txt',
                 names=['Sepal_length', 'Sepal_width', 'Petal_length',
                        'Petal_width', 'clase'],
                 dtype=np.float32)

# Se barajan los ejemplos
df = df.sample(frac=1, random_state=2032).reset_index(drop=True)

# La descripción de los lirios se almacena en 'laX' y la clase en 'laY'
laX = df.iloc[:, :-1]  # todas las columnas salvo las última
laY = df.iloc[:, -1:]  # la última columa es la clase

# En el vídeo donde se habla de la arquitecutra se comentó que es necesario 
# utilizar la codificación one-hot para codificar la clase. Esta codificación
# podemos obtenerla usando la función 'to_categorical' de 'keras.utils'. Consulta 
# la ayuda para entender cómo funciona
laY_one_hot = to_categorical(laY)
print(laX)
print(laY_one_hot)


## 4. Entrenar el modelo
Una vez que  hemos definido cómo va a ser nuestro modelo y que tenemos el conjunto de datos preparado, vamos a entrenarlo para buscar los parámetros que hagan mínima la función de pérdida.

Para ello vamos a llamar a la función **entrena** que definimos anteriormente. Necesita que le suministremos los ejemplos (laX, laY_one_hot), el learning rate, el tamaño del batch y el número de epochs. 



In [0]:
# se entrena
learning_rate = 0.0001  # learning rate
tam_batch = 32   # Tamaño del batch
n_epochs = 1000  # Número de epochs
entrena(laX, laY_one_hot, learning_rate, tam_batch, n_epochs)

##5. Efecto del learning rate
El resultado que se obtiene en esta ejecución no es muy bueno, ya que en la gráfica se ve que el error ha bajado un poco y luego se ha estancado en torno a 1.08.

Esto puede deberse a que el ratio de aprendizaje (learning rate) es demasiado pequeño (consulta el vídeo en el que se habla del descenso del gradiente). Prueba a cambiarlo asignándole el valor 0.001 y vuelve a ejecutar la función. Ahora se ve que la curva está bajando y al alcanzar el número de epochs tiene un error en torno a 0.71. Viendo la forma de la gráfica se ve que todavía podría seguir bajando.

Aumenta nuevamente el learning rate asignándole 0.01 y ejecuta. La curva ahora baja más rápidamente y llegado un punto comienza a oscilar, pero finaliza con una loss mejor.

Si seguimos aumentando el learning rate (0.1)  vemos que baja rápidamente y que hay muchas oscilaciones.

Si probamos con learning rate 1, vemos que el algoritmo no es capaz de alcanzar una buena solución.

Durante el entrenamiento de una red, hay que tener especial cuidado con el valor del learning rate. Será necesario buscar el valor que más se adecúe a nuestro problema y nuestra red.

Sin embargo, que el uso de un learning rate haga que se obtengan mejores o peores resultados, depende también de otros factores, como por ejemplo el número de epochs. Si utilizamos un número de epochs más grande, le daremos más tiempo al algoritmo y, tal vez, valores de learnig rate más bajos vayan haciendo mínimo el error de una manera más lenta y segura.

##6. El tamaño del batch

La evolución del modelo durante el aprendizaje también depende del tamaño del batch. Tamaños de batch más pequeños aceleran el aprendizaje, ya que un tamaño pequeño implica que se harán más actualizaciones en cada epoch. Sin embargo, tamaños de batch demasiado pequeños pueden provocar inestabilidad en el aprendizaje cuando nos enfrentamos a problemas con mucho ruido.

Por ejemplo, con un tamaño de batch 32 y 1000 epochs, sabiendo que tenemos 150 ejemplos, se están haciendo ceil(150/32)x1000=5000 actualizaciones de los parámetros. Si pruebas esta configuración con un learning rate de 0.001 deberías obtener un resultado en torno a 0.71.

Si bajas el tamaño del batch a 16, entonces se harán ceil(150/16)x1000=10000 actualizaciones de los parámetros y el error bajará más rápido (0.47).

Si bajas el tamaño del batch a 8, entonces se harán ceil(150/8)x1000=19000 actualizaciones de los parámetros y el error bajará aún más (0.13).

Si pruebas con batch 2, entonces se harán ceil(150/2)x1000=75000 actualizaciones de los parámetros y el error bajará hasta (0.04).


##7. Más pruebas
Prueba las combinaciones que quieras y analiza los resultados

